# Mesh generation for tutorial 02

In [ ]:
import numpy as np
import pygmsh
from mpi4py import MPI
from dolfinx import cpp
from dolfinx.mesh import create_mesh, create_meshtags
from dolfinx.io import ufl_mesh_from_gmsh, XDMFFile
from dolfinx.plotting import plot

In [ ]:
assert MPI.COMM_WORLD.size == 1, "This mesh generation notebook is supposed to be run in serial"

### Geometrical parameters

In [ ]:
pre_step_length = 4.
after_step_length = 14.
pre_step_height = 3.
after_step_height = 5.
lcar = 1. / 5.

### Create pygmsh mesh

In [ ]:
geom = pygmsh.geo.Geometry()
p0 = geom.add_point([0.0, after_step_height - pre_step_height, 0.0], lcar)
p1 = geom.add_point([pre_step_length, after_step_height - pre_step_height, 0.0], lcar)
p2 = geom.add_point([pre_step_length, 0.0, 0.0], lcar)
p3 = geom.add_point([pre_step_length + after_step_length, 0.0, 0.0], lcar)
p4 = geom.add_point([pre_step_length + after_step_length, after_step_height, 0.0], lcar)
p5 = geom.add_point([0.0, after_step_height, 0.0], lcar)
l0 = geom.add_line(p0, p1)
l1 = geom.add_line(p1, p2)
l2 = geom.add_line(p2, p3)
l3 = geom.add_line(p3, p4)
l4 = geom.add_line(p4, p5)
l5 = geom.add_line(p5, p0)
geom.add_physical([l5], label=1)
geom.add_physical([l0, l1, l2, l4], label=2)
line_loop = geom.add_line_loop([l0, l1, l2, l3, l4, l5])
domain = geom.add_plane_surface(line_loop)
geom.add_physical(domain, label=0)
pygmsh_mesh = pygmsh.generate_mesh(geom)

### Convert to a dolfinx mesh

In [ ]:
cells, x = pygmsh_mesh.cells_dict["triangle"], pygmsh_mesh.points[:, :2]
mesh = create_mesh(MPI.COMM_WORLD, cells, x, ufl_mesh_from_gmsh("triangle", x.shape[1]))

In [ ]:
plot(mesh)

### Extract boundaries as MeshTags

In [ ]:
boundaries_entities = pygmsh_mesh.cells_dict["line"]
boundaries_values = pygmsh_mesh.cell_data_dict["gmsh:physical"]["line"]
mesh.topology.create_connectivity(mesh.topology.dim - 1, 0)
boundaries = create_meshtags(mesh, mesh.topology.dim - 1,
                             cpp.graph.AdjacencyList_int32(boundaries_entities), np.int32(boundaries_values))
boundaries.name = "boundaries"

### Save mesh and boundaries

In [ ]:
with XDMFFile(MPI.COMM_WORLD, "backward_facing_step.xdmf", "w") as file:
    file.write_mesh(mesh)
    mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)
    file.write_meshtags(boundaries)